# 1. 강의영상 

{{<video https://youtu.be/playlist?list=PLQqh36zP38-x6USW3HM9Lm-B19o9qrm19&si=EFy8hdlgDJ-LUFHi >}}

# 2. Import

``` python
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
```

    /kaggle/input/titanic/train.csv
    /kaggle/input/titanic/test.csv
    /kaggle/input/titanic/gender_submission.csv

In [10]:
#pip install autogluon

In [11]:
from autogluon.tabular import TabularDataset, TabularPredictor

# 3. 분석의 절차

## A. 데이터

`-` 비유: 문제를 받아오는 과정으로 비유할 수 있다. 

In [12]:
!kaggle competitions download -c titanic
!unzip titanic.zip -d titanic
df_train = TabularDataset('titanic/train.csv')
df_test = TabularDataset('titanic/test.csv')
!rm titanic.zip
!rm -rf titanic/

  0%|                                               | 0.00/34.1k [00:00<?, ?B/s]
100%|███████████████████████████████████████| 34.1k/34.1k [00:00<00:00, 407kB/s]
Archive:  titanic.zip
  inflating: titanic/gender_submission.csv  
  inflating: titanic/test.csv        
  inflating: titanic/train.csv       


Loaded data from: titanic/train.csv | Columns = 12 / 12 | Rows = 891 -> 891
Loaded data from: titanic/test.csv | Columns = 11 / 11 | Rows = 418 -> 418


## B. Predictor 생성 

`-` 비유: 문제를 풀 학생을 생성하는 과정으로 비유할 수 있다. 

In [19]:
predictr = TabularPredictor("Survived",verbosity=False)

## C. 적합(fit) 

`-` 비유: 학생이 공부를 하는 과정으로 비유할 수 있다. 

`-` 학습

In [20]:
predictr.fit(df_train,presets='best_quality') # 학생(predictr)에게 문제(tr)를 줘서 학습을 시킴(predictr.fit())

/home/cgb2/anaconda3/envs/ag2/lib/python3.11/site-packages/autogluon/tabular/models/fastainn/tabular_nn_fastai.py:200: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)
/home/cgb2/anaconda3/envs/ag2/lib/python3.11/site-packages/autogluon/tabular/models/fastainn/tabular_nn_fastai.py:200: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)
/home/cgb2/anaconda3/envs/ag2/lib/python3.11/site-packages/autogluon/tabular/models/fastainn/tabular_nn_fastai.py:200: FutureWarning: The 'downcast' keyword in fi

`-` 리더보드확인 (모의고사채점)

In [21]:
predictr.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.884400,accuracy,0.642903,172.567142,0.001174,1.079618,2,True,111
1,NeuralNetFastAI_r187_BAG_L1,0.869809,accuracy,0.090789,32.114868,0.090789,32.114868,1,True,104
2,NeuralNetFastAI_r134_BAG_L1,0.865320,accuracy,0.089633,31.923197,0.089633,31.923197,1,True,59
3,CatBoost_r177_BAG_L1,0.864198,accuracy,0.021514,4.640301,0.021514,4.640301,1,True,14
4,NeuralNetFastAI_r69_BAG_L1,0.864198,accuracy,0.051628,5.574065,0.051628,5.574065,1,True,84
...,...,...,...,...,...,...,...,...,...,...
106,ExtraTrees_r126_BAG_L1,0.822671,accuracy,0.103725,0.674517,0.103725,0.674517,1,True,99
107,ExtraTrees_r4_BAG_L1,0.817059,accuracy,0.113228,0.502272,0.113228,0.502272,1,True,66
108,RandomForest_r34_BAG_L1,0.802469,accuracy,0.111451,0.566161,0.111451,0.566161,1,True,60
109,KNeighborsDist_BAG_L1,0.636364,accuracy,0.003525,0.002868,0.003525,0.002868,1,True,2


## D. 예측 (predict)

`-` 비유: 학습이후에 문제를 푸는 과정으로 비유할 수 있다.

`-` training set 을 풀어봄 (predict) $\to$ 점수 확인 

In [22]:
(df_train.Survived == predictr.predict(df_train)).mean()

/home/cgb2/anaconda3/envs/ag2/lib/python3.11/site-packages/autogluon/tabular/models/fastainn/tabular_nn_fastai.py:200: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)
/home/cgb2/anaconda3/envs/ag2/lib/python3.11/site-packages/autogluon/tabular/models/fastainn/tabular_nn_fastai.py:200: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)
/home/cgb2/anaconda3/envs/ag2/lib/python3.11/site-packages/autogluon/tabular/models/fastainn/tabular_nn_fastai.py:200: FutureWarning: The 'downcast' keyword in fi

0.920314253647587

`-` test set 을 풀어봄 (predict) $\to$ 점수 확인 하러 캐글에 결과제출 

In [17]:
# df_test.assign(Survived = predictr.predict(df_test)).loc[:,['PassengerId','Survived']]\
# .to_csv("autogluon(best_quality)_submission.csv",index=False)

# 3. 숙제

`-` 캐글에 제출한 결과를 캡쳐하여 LMS에 제출